# Improving model performance with xfeat, RAPIDS and Optuna


## Introduction

Feature Engineering is the processing of transforming raw data into features that can represent the underlying patterns of the data better. They can help boost the accuracy by a great deal and improve the ability of the model to generalise on unseen data. Every data scientist knows the importance feature engineering. Spending some time thinking about how best to apply and combine the available features can be very meaningful. 

Hyper parameter Optimisation is another such process which can help complement a good model by tuning it's hyperparameters, which can have a tremendous impact on the accuracy of the model. The time and resources required for these processes are generally the reason they're overlooked. 

With xfeat, RAPIDS and Optuna - we aim to bridge these gaps and elevate the performance. 

## What is Optuna?
[Optuna](https://github.com/optuna/optuna) s a lightweight framework for automatic hyperparameter optimization. It provides a define-by-run API, which makes it easy to adapt to any already existing code that we have and enables high modularity and the flexibility to construct hyperparameter spaces dynamically. By simply wrapping the objective function with Optuna can help perform a parallel-distributed HPO search over a search space. As we'll see in this notebook.

## What is xfeat?
[xfeat](https://github.com/pfnet-research/xfeat) is a feature engineering & exploration library using GPUs and Optuna. It provides a scikit-learn-like API for feature engineering with support for pandas and cuDF dataframes and cuPy arrays. 

## What is RAPIDS?
[RAPIDS](https://rapids.ai/about.html) framework  provides a library suite that can execute end-to-end data science pipelines entirely on GPUs.  The libraries in the framework include [cuDF](https://github.com/rapidsai/cudf) - a GPU Dataframe with pandas-like API, [cuML](https://github.com/rapidsai/cuml) - implement machine learning algorithms that provide a scikit-learn-like API and many more. You can learn more [here](https://github.com/rapidsai).

In this notebook, we'll show how one can use these tools together to develop and improve a machine learning model. We'll use Airlines dataset (20M rows) to predict if a flight will be delayed or not. We'll explore how to use Optuna with RAPIDS and the speedups that we can achieve with the integration of these.

In [1]:
from cuml import LogisticRegression
import cudf
import optuna
from cuml.metrics import accuracy_score
from cuml.preprocessing.model_selection import train_test_split
import numpy as np
from xfeat.pipeline import Pipeline
from xfeat.num_encoder import SelectNumerical
from xfeat.selector import ChiSquareKBest
from xfeat.optuna_selector import KBestThresholdExplorer, GroupCombinationExplorer
from cuml import LogisticRegression
from functools import partial
import xfeat
from sklearn.model_selection import KFold

from xfeat import ArithmeticCombinations, Pipeline, SelectNumerical, LabelEncoder, SelectCategorical, TargetEncoder

### Feature Engineering

We'll be using the following functions to perform a few featur engineering tasks on the data. The `feature_engineering` function is called on the dataframe `df`, in this function we perform a simple Arithmetic Combinations on the numerical columns that adds two columns to create a new one. We specify the `operator` and `r` - r is used to indicate how many columns need to be combined.

Then we call `categorical_encoding` which converts the categorical columns to numerical ones and then performs `target_encoding`. Target Encoding replaces the value with the target mean. This is helpful in classification problem to boost the model accuracy. Find more resources at the end of the notebook.

You'll also notice we use `Pipeline` from xfeat to combine two or more feature engineering tasks together. This is useful to concatenate encoders sequentially.

Read more about Feature Encoding and Pipelining with xfeat [here](https://github.com/pfnet-research/xfeat/blob/master/_docs/feature_encoding.md).

In [2]:
def feature_engineering(df, target_col="ArrDelayBinary"):
    """
    Perform feature engineering and return a new df with engineered features
    """
    df_.index = df_.index.rename('original_index')
    df_train, df_test = train_test_split(df, train_size=0.7, random_state=np.random.seed(0), shuffle=True)
    
    # Have to call reset_index for TargetEncoder
    df_train = df_train.reset_index()
    df_test = df_test.reset_index()
    
    # combine into one pipeline
    encoder = Pipeline([
                        LabelEncoder(output_suffix="",
                                    input_cols=["Dest", "Origin", "UniqueCarrier"]),
                        TargetEncoder(target_col=target_col,
                                      input_cols=["Dest", "Origin", "UniqueCarrier"],
                                      output_suffix=""),
                        SelectNumerical(),
                        ArithmeticCombinations(exclude_cols=[target_col, "original_index"],
                                               drop_origin=False,
                                               operator="+",
                                               r=2,
                                               output_suffix="_plus")
                        
                         
    
                    ])
    df_train = encoder.fit_transform(df_train)
    df_test = encoder.transform(df_test)
    
    df = cudf.concat([df_train, df_test])
    df[target_col] = df[target_col].astype('int32')
    df = df.set_index('original_index')
    return df

### Feature Selection and Hyper parameter Optimisation

Now that we have some new features, how do we know they are relevant for the task or represent anything meaningful? We use the feature selection process to do this. This helps in selection of a subset of features that are  most informative. This helps in simplifying the problem and ensures that we aren't overloading the system with unimportant features. Optuna provides a way to choose a `selector` which accepts a `Pipeline` object from xfeat. You can see in the `feature_selection` function we define a Pipeline that takes in an Explorer and a Selection Algorithm (`ChiSquareKBest`). We pass this to an Optuna Study object, along with an Objective function

### Objective Function
The objective function will be the one we optimize in Optuna Study. Objective funciton tries out different values for the parameters that we are tuning and saving the results in `study.trials_dataframes()`.

Let's define the objective function for this HPO task by making use of the `train_and_eval()`. You can see that we simply choose a value for the parameters and call the `train_and_eval` method, making Optuna very easy to use in an existing workflow.

The objective remains constant over different samplers, which are built-in options in Optuna to enable the selection of different sampling algorithms that optuna provides. Some of the available ones include - GridSampler, RandomSampler, TPESampler, etc. We'll use TPESampler for this demo, but feel free to try different samplers to notice the chnages in performance.


### HPO Trials and Study
Optuna uses [study](https://optuna.readthedocs.io/en/stable/reference/study.html) and [trials](https://optuna.readthedocs.io/en/stable/reference/trial.html) to keep track of the HPO experiments. Put simply, a trial is a single call of the objective function while a set of trials make up a study. We will pick the optimal performing trial from a study to get the best parameters that were used in that run.

In [3]:
def train_and_eval(df, penalty='l2', C=1.0, l1_ratio='None', fit_intercept='True', selector=None):

    # Splitting data and prepping for selector fit
    X_train,  X_test, y_train, y_test = train_test_split(df, "ArrDelayBinary",random_state=np.random.seed(0), shuffle=True)

    if selector:
        # For the selector, the label also needs to be in the DF
        X_train["ArrDelayBinary"] = y_train
        X_test["ArrDelayBinary"] = y_test
        
        X_train = selector.fit_transform(X_train)
        X_test = selector.transform(X_test)
    
    # Train and get accuracy
    classifier = LogisticRegression(penalty=penalty,
                                    C=C,
                                    l1_ratio=l1_ratio,
                                    fit_intercept=fit_intercept,
                                    max_iter=10000)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict_proba(X_test.values)[:, 1]
    score = accuracy_score(y_test, y_pred)
    return score

def objective(df, selector, trial):
    """
    Performs the training and evaluation of the set of parameters and subset of features using selector.
    """
    selector.set_trial(trial)
    
    # Select Params
    C = trial.suggest_uniform("C", 0 , 7.0)
    penalty = trial.suggest_categorical("penalty", ['l1', 'none', 'l2'])
    l1_ratio = trial.suggest_uniform("l1_ratio", 0 , 1.0)
    fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
    
    score = train_and_eval(df,
                           penalty=penalty,
                           C=C,
                           l1_ratio=l1_ratio,
                           fit_intercept=fit_intercept,
                          selector=selector)
    return score

def feature_selection(df):
    """
    Defines the Pipeline and performs the optuna opt
    """
    selector = Pipeline(
        [
            SelectNumerical(),
            KBestThresholdExplorer(ChiSquareKBest(target_col="ArrDelayBinary")),
        ]
    )

    df.index = df.index.rename('original_index')
    
    study = optuna.create_study(direction="maximize")
    study.optimize(partial(objective, df, selector), n_trials=N_TRIALS)

    selector.from_trial(study.best_trial)
    selected_cols = selector.get_selected_cols()
    
    return study, df[selected_cols].reset_index().set_index('original_index')

In [4]:
INPUT_FILE = "/home/hyperopt/hyperopt/data/air_par.parquet"

N_ROWS = 10000000
N_TRIALS = 100
df_ = cudf.read_parquet(INPUT_FILE)[:N_ROWS]

df_["ArrDelayBinary"] = df_["ArrDelayBinary"].astype('int32')

# Can't handle nagative values, yet
df_ = df_.drop(["ActualElapsedTime"], axis=1)
# indices = df_.loc[df_["ActualElapsedTime"] < 0].index
# df_.loc[indices, "ActualElapsedTime"] = -1 * df_.loc[indices, "ActualElapsedTime"]

In [5]:
print("Default performance: ", train_and_eval(df_))

Default performance:  0.8123924136161804


In [6]:
# We cast to objects for categorical  and target encoding
df_["UniqueCarrier"] = df_["UniqueCarrier"].astype("object")
df_["Origin"] = df_["Origin"].astype("object")
df_["Dest"] = df_["Dest"].astype("object")

In [7]:
df_feature_eng = feature_engineering(df_)

In [8]:
print("After feature eng: ", train_and_eval(df_feature_eng))

After feature eng:  0.8120155930519104


In [9]:
study, df_select = feature_selection(df_feature_eng)

[I 2020-07-28 21:57:40,560] Finished trial#0 with value: 0.8117660284042358 with parameters: {'C': 2.610316541350379, 'penalty': 'none', 'l1_ratio': 0.1728394675546583, 'fit_intercept': False, 'KBestThresholdExplorer.k': 75.0}. Best is trial#0 with value: 0.8117660284042358.
[I 2020-07-28 22:01:19,766] Finished trial#1 with value: 0.8118404150009155 with parameters: {'C': 5.849864430092992, 'penalty': 'l2', 'l1_ratio': 0.47804617074727584, 'fit_intercept': True, 'KBestThresholdExplorer.k': 42.0}. Best is trial#1 with value: 0.8118404150009155.
[I 2020-07-28 22:04:59,979] Finished trial#2 with value: 0.8115136027336121 with parameters: {'C': 6.7549745715346665, 'penalty': 'l1', 'l1_ratio': 0.35276010044311445, 'fit_intercept': False, 'KBestThresholdExplorer.k': 32.0}. Best is trial#1 with value: 0.8118404150009155.
[I 2020-07-28 22:08:41,742] Finished trial#3 with value: 0.8121348023414612 with parameters: {'C': 3.5824327747346434, 'penalty': 'none', 'l1_ratio': 0.6199295707683428, 'fit

In [10]:
df_select["ArrDelayBinary"] = df_["ArrDelayBinary"]

In [11]:
df_select.columns

Index(['Month', 'DayofWeek', 'CRSDepTime', 'CRSArrTime', 'FlightNum', 'Origin',
       'Dest', 'Distance', 'YearCRSDepTime_plus', 'YearCRSArrTime_plus',
       'YearFlightNum_plus', 'YearDistance_plus', 'MonthDayofWeek_plus',
       'MonthCRSDepTime_plus', 'MonthCRSArrTime_plus',
       'MonthUniqueCarrier_plus', 'MonthFlightNum_plus', 'MonthOrigin_plus',
       'MonthDest_plus', 'MonthDistance_plus', 'MonthDiverted_plus',
       'DayofMonthCRSDepTime_plus', 'DayofMonthCRSArrTime_plus',
       'DayofMonthFlightNum_plus', 'DayofMonthOrigin_plus',
       'DayofMonthDistance_plus', 'DayofWeekCRSDepTime_plus',
       'DayofWeekCRSArrTime_plus', 'DayofWeekUniqueCarrier_plus',
       'DayofWeekFlightNum_plus', 'DayofWeekOrigin_plus', 'DayofWeekDest_plus',
       'DayofWeekDistance_plus', 'DayofWeekDiverted_plus',
       'CRSDepTimeCRSArrTime_plus', 'CRSDepTimeUniqueCarrier_plus',
       'CRSDepTimeFlightNum_plus', 'CRSDepTimeOrigin_plus',
       'CRSDepTimeDest_plus', 'CRSDepTimeDistance_plu

In [12]:
params = study.best_params
print("After feature selection and paramter tuning: ", train_and_eval(df_select,
                                                                      C=params['C'],
                                                                      penalty=params['penalty'],
                                                                      l1_ratio=params['l1_ratio'],
                                                                      fit_intercept=params['fit_intercept']))

After feature selection and paramter tuning:  0.8119720220565796


In [13]:
# study.trials_dataframe().to_csv("xfeat_chi2_100Trials_run3_catenc.csv", header=True)

In [14]:
print(params)

{'C': 3.7590996262114307, 'penalty': 'none', 'l1_ratio': 0.6583132511929572, 'fit_intercept': True, 'KBestThresholdExplorer.k': 61.0}



|Run No.|Default|After FE| Optuna Best| After Selection and HPO|Best Params| Cols|
|-|-|-|-|-|-|-|
|1|0.18747319281101227|0.18743759393692017|0.812965989112854|0.8126764297485352| {'C': 2.7250515887031064,'penalty': 'l2','l1_ratio':0.1403560039741595, 'fit_intercept': True, 'KBestThresholdExplorer.k': 1.0} |['UniqueCarrierFlightNum_plus', 'ArrDelayBinary']|
|2|0.18741360306739807|0.1874103993177414|0.812959611415863|0.8120356202125549|{'C': 3.398344321379011,'penalty': 'none','l1_ratio': 0.5549805266380305,'fit_intercept': True,'KBestThresholdExplorer.k': 1.0}|['FlightNumDistance_plus', 'ArrDelayBinary']|
|3|0.18733720481395721|0.18729199469089508|0.8128544092178345|0.812743604183197|{'C': 4.202429766198819, 'penalty': 'none', 'l1_ratio': 0.6571582748501704, 'fit_intercept': False, 'KBestThresholdExplorer.k': 1.0}|['UniqueCarrierFlightNum_plus', 'ArrDelayBinary']|

## Additional Resources
[How to Win a DS Kaggle competition](https://www.coursera.org/learn/competitive-data-science)

[Target Encoding and Bayesian Target Encoding](https://towardsdatascience.com/target-encoding-and-bayesian-target-encoding-5c6a6c58ae8c)
